In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt

In [2]:
str_files = ''

#df = pd.read_csv(str_files + "ph.csv",encoding='utf-8',sep=';')
dateparse = lambda dates:pd.datetime.strptime(dates,'%d/%m/%Y')
df = pd.read_csv(str_files + 'Fosforo.CSV',encoding='utf-8',sep=';', parse_dates = ['data_coleta'],index_col=['data_coleta'],date_parser=dateparse)

In [3]:
z = df.loc[df['UGRHI'] == '10 - SOROCABA/MEDIO TIETE']

In [4]:
z = z.dropna(axis=1,how='any')

In [5]:
trainingd = z.resample('3M').mean()
trainingd
data = trainingd.iloc[:,1:2].values
train = trainingd.iloc[0:132,1:2].values  
test = trainingd.iloc[132:,1:2].values   
print('treino:',len(train))
print('teste:',len(test))

treino: 132
teste: 34


In [6]:
sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [7]:
def prepare_data(data, lags):
    X, y = [], []
    for row in range(len(data) - lags - 1):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags, 0])
    return np.array(X), np.array(y)  

In [8]:
lags = 1
train_X, train_y = prepare_data(train, lags)
test_X, test_y = prepare_data(test, lags)

train_X = np.reshape(train_X, (train_X.shape[0],1,train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0],1,test_X.shape[1]))
print('Novo formato de treino:')
print(train_X.shape) 

Novo formato de treino:
(130, 1, 1)


In [9]:
model = Sequential() 

model.add(LSTM(units = 10, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1,patience=5)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 16s 122ms/sample - loss: 0.0077 - val_loss: 0.3740
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3682
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3649
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3646
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3646
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3665
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3676
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3677
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3680
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3712
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3699
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3704
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3710
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3724
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3729
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3719
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3722
Epoch 87/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3708
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [10]:
train_score10 = model.evaluate(train_X, train_y, verbose=0)
print(train_score10)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score10, math.sqrt(train_score10)))
test_score10 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score10, math.sqrt(test_score10)))

0.00754388347371647
Pontuação de Treino: 0.01 MSE (0.09 RMSE)
Pontuação de teste: 0.37 MSE (0.61 RMSE)


In [11]:
model = Sequential() 

model.add(LSTM(units = 25, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='min',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 19s 145ms/sample - loss: 0.0078 - val_loss: 0.3731
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3678
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3650
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3631
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3632
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3652
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3671
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0075 - val_loss: 0.3686
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3686
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3749
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3745
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3743
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3744
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3746
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3751
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3751
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3757
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3754
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [12]:
train_score25 = model.evaluate(train_X, train_y, verbose=0)
print(train_score25)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score25, math.sqrt(train_score25)))
test_score25 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score25, math.sqrt(test_score25)))

0.007542712768134572
Pontuação de Treino: 0.01 MSE (0.09 RMSE)
Pontuação de teste: 0.37 MSE (0.61 RMSE)


In [13]:
model = Sequential() 

model.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 16s 123ms/sample - loss: 0.0076 - val_loss: 0.3676
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3630
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3640
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3686
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3702
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3698
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3691
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3692
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3699
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0076 - val_loss: 0.3748
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3761
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3771
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3780
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3764
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3767
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0076 - val_loss: 0.3751
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3742
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0075 - val_loss: 0.3752
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [14]:
train_score50 = model.evaluate(train_X, train_y, verbose=0)
print(train_score50)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score50, math.sqrt(train_score50)))
test_score50 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score50, math.sqrt(test_score50)))

0.0075438987846260155
Pontuação de Treino: 0.01 MSE (0.09 RMSE)
Pontuação de teste: 0.38 MSE (0.61 RMSE)
